### Import libraries

In [ ]:
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'
import random
from glob import glob
import numpy as np
import matplotlib.pyplot as plt
from pylab import imread,subplot,imshow,show
from sklearn.metrics import confusion_matrix, classification_report
import tensorflow
from tensorflow import keras
from tensorflow.keras import optimizers, layers, models, callbacks
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Conv2D, Flatten, MaxPool2D, Dropout
from tensorflow.keras.applications import mobilenet_v2
from tensorflow.keras.preprocessing.image import ImageDataGenerator 

### Train, validation and test directories

In [ ]:
base_dir = 'data/grocery_store_dataset'

train_dir =  os.path.join(base_dir, 'train/Vegetables')
validation_dir =  os.path.join(base_dir, 'val/Vegetables')
test_dir =  os.path.join(base_dir, 'test/Vegetables')

print('Total number of training vegetable images:', sum(len(files) for _, _, files in os.walk(train_dir)))
print('Total number of validation vegetable images:', sum(len(files) for _, _, files in os.walk(validation_dir)))
print('Total number of test vegetable images:', sum(len(files) for _, _, files in os.walk(test_dir)))

vegetable_classes=[]
for root, subdirectories, files in os.walk(train_dir):
    for subdirectory in subdirectories:
        vegetable_classes.append(subdirectory)

### Display dataset images

In [ ]:
multiple_images = glob('grocery_store_dataset/train/Vegetables/Carrots/**')
random_samples = random.sample(multiple_images, 9)

fig, ax = plt.subplots(3, 1,figsize=(20,20))
for i in range(3):
    ax[i].imshow(imread(random_samples[i])), plt.axis('off')

### Data preprocessing

In [ ]:
train_generator = ImageDataGenerator(
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        fill_mode='nearest',
        rescale=1./255,
        zoom_range=0.2,
        horizontal_flip=True)

train_dataset = train_generator.flow_from_directory(
        train_dir,
        target_size=(224, 224),
        batch_size=25,
        class_mode='categorical')

test_dataset = ImageDataGenerator(rescale=1./255).flow_from_directory(
        test_dir,
        target_size=(224, 224),
        batch_size=1,
        shuffle = False,
        class_mode='categorical')

validation_dataset = ImageDataGenerator(rescale=1./255).flow_from_directory(
        validation_dir,
        target_size=(224, 224),
        batch_size=10,
        class_mode='categorical')

### Model configuration

In [ ]:
mobilenet_model = mobilenet_v2.MobileNetV2(weights='imagenet',include_top=False, input_shape=(224, 224, 3)) 
mobilenet_model.trainable = False

model = Sequential()
model.add(mobilenet_model)
model.add(GlobalAveragePooling2D())
model.add(Dropout(0.4))
model.add(Dense(22, activation='softmax'))

model.summary()

model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.Adam(learning_rate=0.001),
              metrics=['accuracy'])

### Fitting the model

In [ ]:
early_stopping = callbacks.EarlyStopping(monitor='val_accuracy', min_delta=0.01, patience= 3)
checkpoint = [
            callbacks.ModelCheckpoint(
            filepath='./vegetables.h5',
            save_best_only=True,
            monitor='val_loss')
]
callbacks=[early_stopping, checkpoint]


history = model.fit(train_dataset,
          batch_size = 36,
          epochs=100,
          validation_data=validation_dataset, callbacks=callbacks)

### Plotting the results

In [ ]:
plt.figure(figsize=(10, 4))

plt.subplot(1, 2, 1)
plt.title('Loss')
plt.plot(np.arange(0, 1), history.history['loss'], label='train')
plt.plot(np.arange(0, 1), history.history['val_loss'], label='val')
plt.legend(loc='best')

plt.subplot(1, 2, 2)
plt.title('Accuracy')
plt.plot(np.arange(0, 1), history.history['accuracy'], label='train')
plt.plot(np.arange(0, 1), history.history['val_accuracy'], label='val')
plt.legend(loc='best')

plt.show()

### Saving the model

In [ ]:
model.save('vegetables_model.h5')

### Prediction

In [ ]:
base_pred = model.predict(test_dataset)
base_pred = base_pred.argmax(axis=1)

print(classification_report(test_dataset.classes, base_pred, target_names=vegetable_classes))